# Dependencies

In [248]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

# Define Foursquare Credentials, Version and Query

In [202]:
CLIENT_ID = 'CZ00R4MCJ00J1D32LVMDFXDMMPFOZHZJBV1KQGITLCRRKZWM' # your Foursquare ID
CLIENT_SECRET = 'DEZUCPKVOBMBDJL1PWO52HUSJP1BF2F2GZAA5C45W0IWLPGY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
latitude = 14.2142
longitude = 120.9687
search_query = "market"
radius = 10000

In [196]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)

#url


results = requests.get(url).json()


In [197]:
venues = results['response']['venues']
venues

[{'id': '4cea2bb20f196dcb133d50ae',
  'name': 'Silang Public Market',
  'location': {'address': 'P.Montoya',
   'crossStreet': 'M.Belen',
   'lat': 14.22332068766597,
   'lng': 120.9716530852005,
   'labeledLatLngs': [{'label': 'display',
     'lat': 14.22332068766597,
     'lng': 120.9716530852005}],
   'distance': 1064,
   'cc': 'PH',
   'city': 'Silang',
   'state': 'Cavite',
   'country': 'Pilipinas',
   'formattedAddress': ['P.Montoya (M.Belen)',
    'Silang',
    'Cavite',
    'Pilipinas']},
  'categories': [{'id': '4bf58dd8d48988d1f7941735',
    'name': 'Flea Market',
    'pluralName': 'Flea Markets',
    'shortName': 'Flea Market',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/fleamarket_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1575031128',
  'hasPerk': False},
 {'id': '4f24b561e4b0dc27b9cb9eee',
  'name': 'Paseo De Sta. Rosa Wet Market',
  'location': {'address': 'Paseo De Sta. Rosa',
   'lat': 14.246371958997296,
   'lng': 12

I couldn't get the temperature as well as the weather data, I'll have to drop that.

# Let's turn the data above as a DataFrame

In [198]:
columns = ['id', 'name', 'latitude', 'longitude']
df = pd.DataFrame();

for data in venues:
    df = df.append({
                        'id'       : data['id'], 
                        'name'     : data['name'],
                        'longitude': data['location']['lng'],
                        'latitude' : data['location']['lat']
                   },
             ignore_index=True)

df = df[['id', 'name', 'latitude', 'longitude']]
df.head()

,id,name,latitude,longitude
0,4cea2bb20f196dcb133d50ae,Silang Public Market,14.223321,120.971653
1,4f24b561e4b0dc27b9cb9eee,Paseo De Sta. Rosa Wet Market,14.246372,121.063700
2,5b5fe292e17910002c453720,Acienda Designer Outlet,14.183884,120.961050
3,4da8b9a4fa8cc76497070fff,Tagaytay Public Market,14.127664,120.980611
4,5283661711d2f7ca94f0e216,Puting Kahoy Wet And Dry Market,14.208261,121.025433


### Let's map the whole dataset.

In [199]:
map_silang = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, name in zip(df['latitude'], df['longitude'], df['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_silang)  
    
map_silang

I wonder how bad the clustering could be here... I'm going to use the k-means clustering where k is 5...

# Get Venue Details for Analysis later on and merge.

In [203]:
columns = ['id', 'likes', 'tips', 'category']
df2 = pd.DataFrame(columns=columns);

for _id in zip(df['id']):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
         
    _id[0],
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION)
    
    results = requests.get(url).json()

    data = results['response']['venue']
    df2 = df2.append({
                        'id'   : _id[0], 
                        'likes': data['likes']['count'],
                        'tips' : data['tips']['count'],
                        'category':data['categories'][0]['name']
                   },
             ignore_index=True)

In [204]:
df = df.merge(df2, how='left', left_on='id', right_on='id')
df.head()

,id,name,latitude,longitude,likes,tips,category
0,4cea2bb20f196dcb133d50ae,Silang Public Market,14.223321,120.971653,6,5,Flea Market
1,4f24b561e4b0dc27b9cb9eee,Paseo De Sta. Rosa Wet Market,14.246372,121.063700,9,1,Fish Market
2,5b5fe292e17910002c453720,Acienda Designer Outlet,14.183884,120.961050,7,5,Outlet Mall
3,4da8b9a4fa8cc76497070fff,Tagaytay Public Market,14.127664,120.980611,27,6,Flea Market
4,5283661711d2f7ca94f0e216,Puting Kahoy Wet And Dry Market,14.208261,121.025433,1,1,Market


In [227]:
df.groupby('category').count()

,id,name,latitude,longitude,likes,tips
category,,,,,,
Boutique,1,1,1,1,1,1
Building,1,1,1,1,1,1
Farmers Market,4,4,4,4,4,4
Fish Market,1,1,1,1,1,1
Flea Market,7,7,7,7,7,7
Flower Shop,1,1,1,1,1,1
Food & Drink Shop,1,1,1,1,1,1
Food Court,2,2,2,2,2,2
Garden Center,1,1,1,1,1,1


In [228]:
# one hot encoding
silang_onehot = pd.get_dummies(df[['category']], prefix="", prefix_sep="")


# move neighborhood column to the first column
fixed_columns = [silang_onehot.columns[-1]] + list(silang_onehot.columns[:-1])
silang_onehot = silang_onehot[fixed_columns]

silang_onehot.head()

,Thrift / Vintage Store,Boutique,Building,Farmers Market,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Garden Center,Grocery Store,Market,Outlet Mall,Plaza,Street Fair
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [230]:
silang_grouped = silang_onehot.groupby(['Farmers Market', 'Market']).mean().reset_index()
silang_grouped

,Farmers Market,Thrift / Vintage Store,Boutique,Building,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Garden Center,Grocery Store,Market,Outlet Mall,Plaza,Street Fair
0,0,0.038462,0.038462,0.038462,0.038462,0.269231,0.038462,0.038462,0.076923,0.038462,0.115385,0.153846,0.038462,0.038462,0.038462
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [256]:
num_top_venues = 5

for hood in silang_grouped['Farmers Market']:
    print("----",hood,"----")
    temp = silang_grouped[silang_grouped['Farmers Market'] == hood].T.reset_index()
    temp.columns = ['Farmers Market', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- 0 ----
           Farmers Market  freq
0             Flea Market  0.27
1                  Market  0.15
2           Grocery Store  0.12
3              Food Court  0.08
4  Thrift / Vintage Store  0.04


---- 1 ----
           Farmers Market  freq
0  Thrift / Vintage Store   0.0
1                Boutique   0.0
2                Building   0.0
3             Fish Market   0.0
4             Flea Market   0.0




In [238]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [242]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Farmers Market']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cat_sorted = pd.DataFrame(columns=columns)
cat_sorted['Farmers Market'] = silang_grouped['Farmers Market']

for ind in np.arange(silang_grouped.shape[0]):
    cat_sorted.iloc[ind, 1:] = return_most_common_venues(silang_grouped.iloc[ind, :], num_top_venues)

cat_sorted.head()

,Farmers Market,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Flea Market,Market,Grocery Store,Food Court,Street Fair,Plaza,Outlet Mall,Garden Center,Food & Drink Shop,Flower Shop
1,1,Street Fair,Plaza,Outlet Mall,Market,Grocery Store,Garden Center,Food Court,Food & Drink Shop,Flower Shop,Flea Market


In [253]:
# set number of clusters
kclusters = 2

cat_clusters = silang_grouped.drop("Farmers Market", 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cat_clusters)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1])